# DS-GA-3001 Advanced Python for Data Science

Before you turn this problem in, make sure you **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart). You can then either run all cells (in the menubar, select Cell$\rightarrow$Run All), or run each cell individually, **in order**, during the class.

Any textual answers that need to be provided will be marked with "YOUR ANSWER HERE". Replace this text with your answer to the question.

Any code answers that need to be provided will be marked with:

```
# YOUR CODE HERE
raise NotImplementedError()
```

Replace all this code with your answer to the question. If you do not answer the question, the `NotImplementedError` exception will be raised, which will indicate to the grader that no answer has been supplied.

In many cases, code answers will also have some associated test code. You should execute the tests after you have entered your code in order to ensure that your answer is correct. You should not proceed to the next question until your answer is correct.

Finally, insert your name and any collaborators in the cell below.

In [1]:
NAME = "Jiayi Lu (jl6583)"
COLLABORATORS = ""

---

# Python Performance Tuning

## Profiling

In order to optimize a program, it is essential to understand where the bottlenecks are. These are the places where the program is spending most of its time. A *profile* is a set of statistics that describes how often and for how long various parts of the program executed.

*Deterministic profiling* is meant to reflect the fact that all function call, function return, and exception events are monitored, and precise timings are made for the intervals between these events (during which time the user’s code is executing). This is in contrast to *statistical profiling* which randomly samples the effective instruction pointer, and deduces where time is being spent. The latter technique traditionally involves less overhead (as the code does not need to be instrumented), but provides only relative indications of where time is being spent.

In Python, since there is an interpreter active during execution, the presence of instrumented code is not required to do deterministic profiling. Python automatically provides a hook (optional callback) for each event. In addition, the interpreted nature of Python tends to add so much overhead to execution, that deterministic profiling tends to only add small processing overhead in typical applications. The result is that deterministic profiling is not that expensive, yet provides extensive run time statistics about the execution of a Python program.

Call counts (i.e. how many times a function is called) and profiling statistics can be used for a variety of purposes:
1. Unusual count numbers can help identify bugs in code.
2. High call counts can help to identify possible points where in-lining (unwrapping loops) might benefit. 
3. Internal time statistics can be used to identify “hot loops” that should be carefully optimized. 
4. Cumulative time statistics can be used to identify high level errors in the selection of algorithms.

Python has two standard modules that provide the same profiling interface: `cProfile` and `profile` (the hotshot module is no longer maintained). The `cProfile` module has the lowest overhead, but because it is written in C, may not be as widely available. The `profile` module is written in Python, so has a much higher overhead, but is easier to extend. There is also a newer module, called `line_profiler` that profiles on a line-by-line basis. This module is not part of the standard distribution, so needs to be installed separately.

## `cProfile`

In [2]:
import cProfile
import re
cProfile.run('re.compile("foo|bar")')

         195 function calls (190 primitive calls) in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 re.py:192(compile)
        1    0.000    0.000    0.000    0.000 re.py:230(_compile)
        1    0.000    0.000    0.000    0.000 sre_compile.py:228(_compile_charset)
        1    0.000    0.000    0.000    0.000 sre_compile.py:256(_optimize_charset)
        1    0.000    0.000    0.000    0.000 sre_compile.py:433(_compile_info)
        2    0.000    0.000    0.000    0.000 sre_compile.py:546(isstring)
        1    0.000    0.000    0.000    0.000 sre_compile.py:552(_code)
        1    0.000    0.000    0.000    0.000 sre_compile.py:567(compile)
      3/1    0.000    0.000    0.000    0.000 sre_compile.py:64(_compile)
        5    0.000    0.000    0.000    0.000 sre_parse.py:137(__len__)
       12    0.000

The first line indicates the number of calls were monitored. Of those calls, some were primitive, meaning that the call was not induced via recursion. The next line, **"`Ordered by: standard name`"**, indicates that the text string in the last column was used to sort the output. The column headings (from left to right) are:

|Heading|Description|
|:---------|:----------|
|`ncalls`|for the number of calls|
|`tottime`|for the total time spent in the given function (and excluding time made in calls to sub-functions)|
|`percall`|is the quotient of `tottime` divided by `ncalls`|
|`cumtime`|is the cumulative time spent in this and all subfunctions (from invocation till exit). This figure is accurate *even* for recursive functions|
|`percall`|is the quotient of `cumtime` divided by primitive calls|
|`filename:lineno(function)`|provides the respective data of each function|

When there are two numbers in the first column (for example 3/1), it means that the function recursed. The second value is the number of primitive calls and the former is the total number of calls. Note that when the function does not recurse, these two values are the same, and only the single figure is printed.

From IPython, the same result can be achieved by using the `%prun` magic command or `%%prun` cell magic command.

In [3]:
import re
%prun re.compile("foo|bar")

It's also possible to run a program under the profiler using the `%run` magic command.

`%run -p [prof_opts] filename.py [args to program]`

From the command line, the cProfile module can be used as follows.

`python -m cProfile [-o output_file] [-s sort_order] filename.py` -- This is how you run cProfile from cmd

### Using cProfile

As a toy example, we would like to evaluate the summation of the reciprocals of squares up to a certain integer $n$ for evaluating $\pi$. The relation we want to use has been proven by Euler in 1735 and is known as the Basel problem.

$\pi^2 = 6 \sum_{k=1}^{\infty} \frac{1}{k^2} = 6 \lim_{k \to \infty} \big( \frac{1}{1^2} + \frac{1}{2^2} + \dots + \frac{1}{k^2} \big) \approx 6 \big( \frac{1}{1^2} + \frac{1}{2^2} + \dots + \frac{1}{n^2} \big)$

A simple Python function for evaluating the truncated sum looks like this:



In [4]:
def recip_square(i):
    return 1./i**2

def approx_pi(n=10000000):
    val = 0.
    for k in range(1,n+1):
        val += recip_square(k)
    return (6 * val)**.5

<div class="alert alert-success">
First, start by timing how long it takes to evaluate the function using the default value of n.
</div>

In [5]:
%timeit approx_pi()

1 loops, best of 3: 2.36 s per loop


Next, profile the code using the `%prun` magic command:

In [6]:
%prun approx_pi()

The first line of the profile contains the number of CPU seconds it took to run the code. You should see that the code was slower than the first run. This was because it ran inside the cProfile module. 

Looking at the `tottime` column, we can see that approximately half the time is spent in `approx_pi` and the other half is spent in `recip_square`. Also, we can see that considerable time is spent in the `range` function.

Using information from the Python Performance Tips module, we know that there is considerable overhead in a function call, so let's try the same code without the extra function.

In [7]:
def approx_pi2(n=10000000):
    val = 0.
    for k in range(1,n+1):
        val += 1./k**2
    return (6 * val)**.5

In [8]:
%prun approx_pi2()

<div class="alert alert-success">
Examining the output from this command, you should see that a significant portion of time is still being spent in one function. Using your knowledge from the Python Performance Tips module, modify the code to reduce or eliminate this overhead. Enter the new code in the cell below.
</div>

In [9]:
from itertools import *
import operator
def approx_pi3(n=10000000):
    val = 0.
    val = sum(imap(operator.pow, xrange(1,n+1), repeat(-2))) #use sum and imap to reduce to function call overhead and for loop overhead
    return (6 * val)**.5

<div class="alert alert-success">
Now time how long it takes to run your version of the function.
</div>

In [10]:
%prun approx_pi3()

<div class="alert alert-success">
Enter the speedup you achieved in the cell below using the equation $speedup = old time / new time$. E.g. if the old time was 2.21s and the new time was 340ms, then the speedup would be $2210 / 340$ or 6.5 times.
</div>

In [11]:
1.26

1.26

## `line_profiler`

The cProfile module shows how much time is being spent in each function. This is a good first step for locating hotspots in a program, and is frequently all that is needed to optimize the code. However, sometimes the cause of the hotspot is actually a single line in the function, and that line may not be obvious from just reading the source code. These cases are particularly frequent in scientific computing. Functions tend to be larger (sometimes because of legitimate algorithmic complexity, sometimes because the programmer is still trying to write FORTRAN code), and a single statement without function calls can trigger lots of computation when using libraries like NumPy. cProfile only times explicit function calls, not special methods called because of syntax. Consequently, a relatively slow NumPy operation on large arrays like this,

```
a[large_index_array] = some_other_large_array
```

is a hotspot that never gets explicitly shown by cProfile because there is no function call in that statement.

The `line_profiler` can be given functions to profile, and it will time the execution of each individual line inside those functions. In a typical workflow, it is only useful to examine the line timings of a few functions because wading through the results of timing every single line of code would be overwhelming. 

The line profiler can be used from IPython by loading the `line_profiler` extension. Run the following command to load the extension.

In [12]:
%load_ext line_profiler

The following code computes the list of prime numbers up to and including $n$ using the Sieve of Eratosthenes, which can be expressed in pseudocode as follows:

**Input**: an integer $n > 1$
 
Let $A$ be an array of Boolean values, indexed by integers $2$ to $n$,
initially all set to **true**.

```
for i = 2, 3, 4, ..., not exceeding sqrt(n):
 if A[i] is true:
  for j = i**2, i**2+i, i**2+2i, i**2+3i, ..., not exceeding n:
    A[j] := false
```

**Output**: all $i$ such that $A[i]$ is **true**.

In [13]:
def primes(n=1000): 
    if n == 2:
        return [2]
    elif n < 2:
        return []
    s = range(3, n + 1, 2)
    mroot = n ** 0.5
    half = (n + 1) / 2 - 1
    i = 0
    m = 3
    while m <= mroot:
        if s[i]:
            j = (m * m - 3)/2
            s[j] = 0
            while j < half:
                s[j] = 0
                j += m
        i = i + 1
        m = 2 * i + 3
    return [2] + [x for x in s if x]

Once you have loaded this function, you can profile it with the line profiler using the following command. The "`-f`" option tells `%lprun` which function to profile.

In [14]:
%lprun -f primes primes(10000)

The line profiler will display some information about the execution, including the line "Timer unit:" which gives the conversion factor to seconds for time information. It then shows a table with the following column headings (from left to right):

|Heading|Description|
|:---------|:----------|
|`Line #`|The line number in the code|
|`Hits`|The number of times that line was executed|
|`Time`|The total amount of time spent executing the line in the timer's units|
|`Per Hit`|The average amount of time spent executing the line once in the timer's unit|
|`% Time`|The percentage of time spent on that line relative to the total amount of recorded time spent in the function|
|`Line Contents`|The actual source code of the line|




From the output, you can see that most of the time was spent at lines 15-17 and line 20. If we are to improve the performance of this function, then we should focus on these lines.

As we have seen previously, one way of improving performance is to use builtin functions rather than Python code. Another way is to use a fast library such as NumPy to replace Python code.

<div class="alert alert-success">
Using one or more of these techniques, write a new function called `faster_primes` that achieves better performance the the original function. Add your code to the cell below, and use the test cell to check the results.
<div>

In [15]:
import numpy
from numpy import arange, ones, where
def faster_primes(n=1000):
    """Fast implementation of the Sieve of Eratosthenes to compute prime numbers. Returns
    a list of the first primes that are not greater than n."""
    primes = [2]
    appendf = primes.append
    seive = ones(n/2, dtype=bool) #create an array that stores only odd numbers from 3 to n
    for i in xrange(0,int(n**0.5)/2): #we only need to iterate to n**0.5/2 since the actual factor we are testing is 2*i+1
        if seive[i]:
            seive[2*i*(i+3)+3::2*(i+1)+1] = False
    idx = 2*where(seive)[0]+3
    return primes + list(idx)

Run the following tests to ensure that your code is correct and results in a performance improvement.

In [16]:
from nose.tools import assert_equal, assert_less
import timeit
assert_equal(primes(1000), faster_primes(1000))
assert_less(timeit.timeit(faster_primes), timeit.timeit(primes))